In [25]:
import pandas as pd
import numpy as np
import decimal
import math as m
from matplotlib import pyplot as plt
import os
import time
import json

np.set_printoptions(formatter={'float': lambda x: "{0:0.8f}".format(x)})

# Reader score must be set to a very small value otherwise there will be a division by 0

epsilon = 0.000001

# CSV file parsing

dataset_name = "ground_truth_2"
dataset_folder_path = "../data/{}/".format(dataset_name)
info_filename = "{}info.csv".format(dataset_folder_path)
ratings_filename = "{}ratings.csv".format(dataset_folder_path)
authors_filename = "{}authors.csv".format(dataset_folder_path)

info = pd.read_csv(info_filename)
paper_authors = pd.read_csv(authors_filename)
paper_authors = paper_authors.values
paper_ratings = pd.read_csv(ratings_filename)
paper_ratings = paper_ratings.values

# Initial setup

dataset_name = info["Dataset"][0]
papers_number = info["Paper"][0]
readers_number = info["Reader"][0]
ratings_number = info["Rating"][0]
authors_number = info["Author"][0]
papers = np.arange(papers_number)
readers = np.arange(readers_number)
ratings = np.arange(ratings_number)
authors = np.arange(authors_number)
paper_steadiness = np.zeros(papers_number,dtype=np.float16)
paper_score = np.zeros(papers_number,dtype=np.float16)
rating_goodness = np.zeros(ratings_number,dtype=np.float16)
reader_steadiness = np.zeros(readers_number,dtype=np.float16)
reader_score = np.zeros(readers_number,dtype=np.float16)
reader_score.fill(epsilon)
author_steadiness = np.zeros(authors_number, dtype=np.float16)
author_score = np.zeros(authors_number, dtype=np.float16)

start_time = time.time()

def get_author(current_paper) :
    
    found_authors = []
    
    for author_index, author_entry in enumerate(paper_authors) :
        current_author = int(author_entry[0])
        written_papers = author_entry[1].split(";")
        written_papers = [int(x) for x in written_papers]
        if current_paper in written_papers :
            found_authors.append(current_author)
            
    return np.asarray(found_authors)

# Function to output result to file

def serialize_result(verbose):
    
    result_folder_path = "../models/{}/".format(dataset_name)

    # Quantities output handling

    dictionary = [
        {'Quantity': 'Paper Steadiness', 'Identifiers': papers.tolist(), 'Values': paper_steadiness.tolist()},
        {'Quantity': 'Paper Score', 'Identifiers': papers.tolist(), 'Values': paper_score.tolist()},
        {'Quantity': 'Reader Steadiness', 'Identifiers': readers.tolist(), 'Values': reader_steadiness.tolist()},
        {'Quantity': 'Reader Score', 'Identifiers': readers.tolist(), 'Values': reader_score.tolist()},
        {'Quantity': 'Author Steadiness', 'Identifiers': authors.tolist(), 'Values': author_steadiness.tolist()},
        {'Quantity': 'Author Score', 'Identifiers': authors.tolist(), 'Values': author_score.tolist()},
    ]
    
    quantities_filename = "{}readersourcing/quantities.json".format(result_folder_path)
    os.makedirs("{}readersourcing/".format(result_folder_path), exist_ok=True)
    
    if verbose:
        print("PRINTING QUANTITIES TO .JSON FILE AT PATH {}".format(quantities_filename))
    
    with open(quantities_filename, 'w') as outfile:  
        json.dump(dictionary, outfile)
        
    # Rating and goodness matrix output handling
    
    rating_matrix = np.zeros((readers_number, papers_number))
    goodness_matrix = np.zeros((readers_number, papers_number))
    
    for index, entry in enumerate(paper_ratings) :
    
        # Example: <1,1,2,0.8,0>
        # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8
        timestamp = int(entry[0])
        reader = int(entry[1])
        paper = int(entry[2])
        rating = entry[3]
        
        rating_matrix[reader][paper] = rating
        goodness_matrix[reader][paper] = rating_goodness[timestamp]
    
    ratings_filename = "{}readersourcing/ratings.csv".format(result_folder_path)
    os.makedirs("{}readersourcing/".format(result_folder_path), exist_ok=True)
    
    goodness_filename = "{}readersourcing/goodness.csv".format(result_folder_path)
    os.makedirs("{}readersourcing/".format(result_folder_path), exist_ok=True)
    
    if verbose:
        print("PRINTING RATING MATRIX TO .CSV FILE AT PATH {}".format(ratings_filename))
            
    np.savetxt(ratings_filename, rating_matrix, delimiter=",")
    
    if verbose:
        print("PRINTING RATING GOODNESS MATRIX TO .CSV FILE AT PATH {}".format(goodness_filename))
    
    np.savetxt(goodness_filename, goodness_matrix, delimiter=",")
    
    # Info output handling
    
    elapsed_time = time.time() - start_time 
    
    info_filename = "{}readersourcing/info.json".format(result_folder_path)
    os.makedirs("{}readersourcing/".format(result_folder_path), exist_ok=True)
    
    dictionary = [{'Time': elapsed_time}]
    
    if verbose:
        print("PRINTING INFO TO .JSON FILE AT PATH {}".format(info_filename))
        
    with open(info_filename, 'w') as outfile:  
        json.dump(dictionary, outfile)
    
    return elapsed_time

# There are many "print" that you can uncomment if you have to do some debugging
# print("##########")

for index, entry in enumerate(paper_ratings) :

    # Example: <1,1,2,0.8,0>
    # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8
    timestamp = int(entry[0])
    reader = int(entry[1])
    paper = int(entry[2])
    rating = entry[3]
    authors_of_paper = get_author(paper)
    
    percentage = 100*index/ratings_number
    if percentage % 2 == 0:
        print("{}/{} ({}/100%)".format(int(index), ratings_number, int(percentage)))
    # print("---------- CURRENT ENTRY ----------")
    # print(f"TIMESTAMP {timestamp} - READER {reader} - PAPER {paper} - SCORE {rating}")
    
    serialize_result(verbose=False)

    # COMPUTATION START: PAPER AND READER SCORE

    # Saving values at time t(i)

    old_paper_steadiness = paper_steadiness[paper]
    old_paper_score = paper_score[paper]
    old_reader_steadiness = reader_steadiness[reader]
    old_rating_goodness = rating_goodness[timestamp]
    old_reader_score = reader_score[reader]
    
    # print("---------- PRINTING VALUES AT TIME T(I) ----------")
    # print("PAPER STEADINESS T(I) ", old_paper_steadiness)
    # print("PAPER SCORE T(I) ", old_paper_score)
    # print("READER STEADINESS T(I) ", old_paper_score)
    # print("RATING GOODNESS T(I) ", rating_goodness[timestamp])
    # print("READER SCORE T(I) ", old_reader_score)

    # Updating values at time t(i+1)

    paper_steadiness[paper] = old_paper_steadiness + old_reader_score
    paper_score[paper] = ((old_paper_steadiness * old_paper_score) + (old_reader_score * rating)) / paper_steadiness[paper]
    rating_goodness[timestamp] = (1 - (m.sqrt(abs(rating - paper_score[paper]))))
    reader_steadiness[reader] = (old_reader_steadiness + paper_steadiness[paper])
    reader_score[reader] = (((old_reader_steadiness * old_reader_score) + (paper_steadiness[paper] * rating_goodness[timestamp])) / reader_steadiness[reader])

    # print("---------- PRINTING VALUES AT TIME T(I+1) ----------")
    # print("PAPER STEADINESS T(I+1) ", paper_steadiness[paper])
    # print("PAPER SCORE T(I+1) ", paper_score[paper])
    # print("READER STEADINESS T(I+1) ", reader_steadiness[reader])
    # print("RATING GOODNESS T(I+1) ", rating_goodness[timestamp])
    # print("READER SCORE T(I+1) ", reader_score[reader])

    # COMPUTATION START: AUTHOR SCORE

    for author in authors_of_paper :
        # Saving values at time t(i)

        old_author_steadiness = author_steadiness[author]
        old_author_score = author_score[author]

        # Updating values at time t(i+1)7

        author_steadiness[author] = old_author_steadiness + old_reader_score
        author_score[author] = ((old_author_steadiness * old_author_score) + (old_reader_score * rating)) / author_steadiness[author]

    # COMPUTATION START: PROPAGATING CHANGES TO PREVIOUS READERS

    if index > 0: 
      
        previous_ratings = paper_ratings[:index]
        previous_paper_ratings = previous_ratings[
           (previous_ratings[:,1]!=float(reader)) &
           (previous_ratings[:,2]==float(paper))
        ]
        
        # print(" ----- PREVIOUS PAPER RATINGS -----")

        for previous_index, previous_entry in enumerate(previous_paper_ratings):
            
            # Example: <1,1,2,0.8,0>
            # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8 written by Author 0
            previous_timestamp = int(previous_entry[0])
            previous_reader = int(previous_entry[1])
            previous_paper = int(previous_entry[2])
            previous_rating = previous_entry[3]

            # print(f"PREVIOUS TIMESTAMP {previous_timestamp} - PREVIOUS READER {previous_reader} - PREVIOUS PAPER {previous_paper} - PREVIOUS RATING {previous_rating}")

            # Saving previous values at time t(i)

            old_previous_reader_steadiness = reader_steadiness[previous_reader]
            old_previous_reader_score = reader_score[previous_reader]
            old_previous_rating = previous_rating
            old_previous_rating_goodness = rating_goodness[previous_timestamp]

            # Updating previous values at time t(i+1)

            rating_goodness[previous_timestamp] = 1 - (m.sqrt(abs(old_previous_rating - paper_score[paper])))
            reader_steadiness[previous_reader] = (old_previous_reader_steadiness + old_reader_score)
            reader_score[previous_reader] = (
                                                (old_previous_reader_steadiness * old_previous_reader_score) -
                                                (old_paper_steadiness * old_previous_rating_goodness) +
                                                (paper_steadiness[paper] * rating_goodness[previous_timestamp])
                                            ) / reader_steadiness[previous_reader]

        # print(" ----- PREVIOUS PAPER RATINGS END -----")
        
    # print("---------- PRINTING FINAL VALUES AT TIME T(I+1) ----------")
    # print("PAPER STEADINESS: ", paper_steadiness)
    # print("PAPER SCORE: ", paper_score)
    # print("READER STEADINESS: ", reader_steadiness)
    # print("READER SCORE: ", reader_score)
    # print("##########")

print("{}/{} (100/100%)".format(int(ratings_number), int(ratings_number))) 
elapsed_time = serialize_result(verbose=True)
print("ELAPSED TIME: ", elapsed_time)


0/6 (0/100%)
3/6 (50/100%)
6/6 (100/100%)
PRINTING QUANTITIES TO .JSON FILE AT PATH ../models/ground_truth_2/readersourcing/quantities.json
PRINTING RATING MATRIX TO .CSV FILE AT PATH ../models/ground_truth_2/readersourcing/ratings.csv
PRINTING RATING GOODNESS MATRIX TO .CSV FILE AT PATH ../models/ground_truth_2/readersourcing/goodness.csv
PRINTING INFO TO .JSON FILE AT PATH ../models/ground_truth_2/readersourcing/info.json
ELAPSED TIME:  0.43799614906311035


In [26]:
# Summary


print("PAPER STEADINESS:  ", paper_steadiness)
print("PAPER SCORE:       ", paper_score)
print("READER STEADINESS: ", reader_steadiness)
print("READER SCORE:      ", reader_score)
print("AUTHOR STEADINESS: ", author_steadiness)
print("AUTHOR SCORE:      ", author_score)

PAPER STEADINESS:   [0.00000304 2.71484375]
PAPER SCORE:        [0.54980469 0.45385742]
READER STEADINESS:  [2.71484375 2.71484375 2.71484375]
READER SCORE:       [0.69384766 0.31005859 0.34887695]
AUTHOR STEADINESS:  [2.71484375 2.71484375]
AUTHOR SCORE:       [0.45385742 0.45385742]
